<center><h2>CS456 Modern Cyber-Security<br /></center><h1><center>Jupyter Lab #1 <br />Demonstration of the SIGNAL Protocol </center></h1>

## Lab Overview

This lab gives you an opportunity to look at the internals the SIGNAL protocol and its use of X3DH and
the double-ratchet mechanism.   The code is a simplified Python duplication of the relevant SIGNAL components.

Most of this lab requires you to simply examine and execute the python cells to see what kind of output is produced. Feel free to change crypto keys variables to see what happens.

The very last exercise will require you to write a tiny amount of original code in order to encrypt a new message and verify that the ratchet values did indeed "click" to a new state.

### Prerequisites

This jupyter notebook can be run on your own laptop or on a CS lab machine or by means of SSH tunnel to a CS lab machine to view the notebook on your local laptop.

All of the python libraries and command dependencies to successfully run this lab should already be installed and available on the CS lab machines. If a library will not import, you should use the --user option in pip to install the library to your own userid. Example:

    pip3 install --user cryptography==2.8 pycrypto

Yo can find guidlines on how to setup your CS lab machine in the link below:

    https://sna.cs.colostate.edu/software/jupyter/

If you are running on your own laptop, you should install the necessary libraries yourself using the pip3 command as described later in this lab. I recomnad to setup Anconda to your machine,for example use the link below to install it on mac:

    https://docs.anaconda.com/anaconda/install/mac-os/
 

### Attribution

This lab is inspired by the web article created by Nikos Filippakis at https://nfil.dev/coding/encryption/python/double-ratchet-example/.   Most of the code and text is leveraged directly from the article, formatted into jupyer notebook format, and some edits and comments from myself.  Additional info is cited from
   https://blog.cloudboost.io/demystifying-the-signal-protocol-for-end-to-end-encryption-e2ee-3e31830c456f.

### What to Submit

You will be instructed at the end of the lab what information you need to submit to Canvas.


----
## Install Python libraries needed by this jupyter notebook

In [39]:
# if the import statements used later do not work, you must install the libraries by executing this cell.  
# some platforms may require you to execute these command in a terminal window instead of this jupyter notebook.
! pip3 install --user cryptography==2.8 pycrypto



## Part 1:  Initialization of Alice and Bob objects and the X3DH exchange

Execute (shift-enter) each of the code cells below and observe the result.  Note, however, that many of the cells do not print any output but simply initialize variables or define class structures.  

In [6]:
# import the libraries used by this notebook; if something doesn't import, correct the issue before moving on.

import base64

from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed25519 import \
        Ed25519PublicKey, Ed25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend

from Crypto.Cipher import AES

### Create the class structures that define "Alice" and "Bob".  

For now, the class structures only contain a few variable and the function to do X3DH.  We will build up these class functions with additional capabilities in future cells.  Notice that this follows my lecture on what key pairs are defined by Bob (Identity key, Signed Prekey, and an operational key) as well as by Alice (her identity key and ephemeral key).  Also note that these use the elliptic curve based on the X25519 parameters.

This cell also defines the b64 and hkdf utility functions.

This jupyter notebook simplifies things by not using an explicit server object for message and key transfers.  Instead we will simply pass variables to each of the objects directly.  This makes the code easier to understand but is obviously not an asynchronous messaging system!

In [7]:
def b64(msg):
    # base64 encoding helper function
    return base64.encodebytes(msg).decode('utf-8').strip()

def hkdf(inp, length):
    # use HKDF on an input to derive a key
    hkdf = HKDF(algorithm=hashes.SHA256(), length=length, salt=b'',
                info=b'', backend=default_backend())
    return hkdf.derive(inp)

class Bob(object):
    def __init__(self):
        # generate Bob's keys
        self.IKb = X25519PrivateKey.generate()
        self.SPKb = X25519PrivateKey.generate()
        self.OPKb = X25519PrivateKey.generate()
        
    def __repr__(self): 
        #provides a way to print the public key values in base64 format.
        IKb_public_bytes = self.IKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        SPKb_public_bytes = self.SPKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        OPKb_public_bytes = self.OPKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        return "Bob Object:\n  Public Identity Key: %s \n  Public Signed Pre-Key: %s \n  Public Operational Key %s" % \
                (b64(IKb_public_bytes), b64(SPKb_public_bytes), b64(OPKb_public_bytes))


    def x3dh(self, alice):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        dh3 = self.SPKb.exchange(alice.EKa.public_key())
        dh4 = self.OPKb.exchange(alice.EKa.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Bob]\tShared key:', b64(self.sk))


class Alice(object):
    def __init__(self):
        # generate Alice's keys
        self.IKa = X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()
        
    def __repr__(self): 
        #provides a way to print the public key values in base64 format.
        IKa_public_bytes = self.IKa.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        EKa_public_bytes = self.EKa.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        return "Alice Object:\n  Public Identity Key: %s \n  Public Ephemeral: %s \n" % \
                (b64(IKa_public_bytes), b64(EKa_public_bytes))

    def x3dh(self, bob):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.SPKb.public_key())
        dh4 = self.EKa.exchange(bob.OPKb.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Alice]\tShared key:', b64(self.sk))
        



### Instantiate the objects for Bob and Alice

Then print out the public key values for each key generated.  It is possible to print the private key as well, but I was too lazy.

Then perform the X3DH operations for both Alice and Bob and print out the derived shared keys and observe that they are identical.  YAY!!! 

Execute this cell several times to see that even though the keys are changing the shared secrets are always equal to each other.

In [8]:
# only the INIT code gets executed when instantiating an object
# If you execute this cell several times you will see that different keys are generated each time. 

alice = Alice()
bob = Bob()

print (alice)
print (bob)
print ()

# Alice performs an X3DH while Bob is offline, using his uploaded keys
alice.x3dh(bob)

# Bob comes online and performs an X3DH using Alice's public keys
bob.x3dh(alice)

Alice Object:
  Public Identity Key: uRVY/XlNUNPYYZNQxnBiwfNvuuqstjGT9bx9YbNVwhM= 
  Public Ephemeral: c22GPWJY/wWGuaGOxu/HPfRTbOK+P6Gq0eijOo8t5Hg= 

Bob Object:
  Public Identity Key: cbjjBqqI80x032UOueHSw0GNQIPzYSlygbY4wetfcFY= 
  Public Signed Pre-Key: 0flTCCybVHC2BmcdIHvti6X1OqBToviDh3fHSH8D2CI= 
  Public Operational Key w35cnBD4X7CqNYGetVbMswdAOruIwzZ8R+4d42FDBgM=

[Alice]	Shared key: FSsjugwP44In1C6wCYQbnifbCQ2YsQ118m3VBFx7bU8=
[Bob]	Shared key: FSsjugwP44In1C6wCYQbnifbCQ2YsQ118m3VBFx7bU8=


## Part 2:  The Double Ratchet Mechanism

### Part 2a:  the send and receive ratchets

We will now create a new class that defines the Message Ratchet.  We will also rebuild the Bob and Alice classes to add a function that initializes their send and receive ratchets.

The symmetric ratchet is implemented with a KDF chain using the HKDF algorithm, which ensures that the output will be cryptographically secure.

On the initial step, the KDF function is supplied with a secret key and some input data, which can be a constant. The output of the KDF is another secret key. This new key is split into two parts: The next KDF key and the message key. This is a turn of the “ratchet”: The internal state of the ratchet (the KDF key) is changed and a new message key is created.

<img src="http://www.cs.colostate.edu/~cs456/images/symmRatchet.png" width="350px" alt="ratchet" />

Because the output of the KDF algorithm is cryptographically secure, it’s hard to reconstruct the input key of the KDF given the output key. This means that an attacker can’t reconstruct older keys even if the current state and message key is leaked. They can however decrypt a single message by using the message key. In addition, by changing the input parameter on each step, we are also guaranteed break-in recovery: An attacker can’t deduce the next state of the ratchet by only knowing the current state if they don’t know what the input is. If the input is constant however an attacker can sync with the ratchet and decrypt all future messages.

Having performed the X3DH algorithm, both Alice and Bob have now arrived at a common shared secret key. That is now used to establish their session keys by using the Double Ratchet algorithm.

Each of them maintains three symmetric ratchets in order to be able to communicate. The first one is the root ratchet. This ratchet is initialized with the shared key of Alice and Bob. The input of this ratchet can be assumed to be constant for now, which however does not provide break-in recovery. This will change in the next section.

The other two ratchets are the sending and receiving ratchets. Alice’s sending ratchet’s state must always match Bob’s receiving ratchet state, and vice-versa. These two ratchets are both initialized from the first two keys provided by the root chain. When Alice wants to send a message to Bob, she turns her sending ratchet once, obtaining a new message key. She then encrypts her message using that message key.

<img src="http://www.cs.colostate.edu/~cs456/images/ratchetPairs.png" width="350px" alt="message ratchets" />

Similarly, Bob initializes his sending and receiving ratchets by turning his root ratchet twice. When he receives a message from Alice he turns his receiving ratchet once, matching the state of Alice’s sending ratchet. This provides him with the key to decrypt the message. He can send messages to Alice in a similar fashion, using his sending ratchet and Alice’s receiving ratchet.

By having two separate ratchets for sending and receiving, we make sure that Bob and Alice won’t have an issue of both claiming the same key from their ratchets and sending each other a message at the same time. Each message is also accompanied by the order that it was sent. This way, if Bob receives a message out of order, he can turn his receiving ratchet more than once to get the appropriate message key to decrypt it. He also stores the message keys that he skipped in case these messages arrive, so that he can then decrypt them.

Let’s add a simple ratchet implementation to our code, and the initialization methods to Bob and Alice.


In [9]:
class SymmRatchet(object):
    def __init__(self, key):
        self.state = key

    def next(self, inp=b''):
        # turn the ratchet, changing the state and yielding a new key and IV
        output = hkdf(self.state + inp, 80)
        self.state = output[:32]
        outkey, iv = output[32:64], output[64:]
        return outkey, iv

In [10]:
class Bob(object):
    def __init__(self):
        # generate Bob's keys
        self.IKb = X25519PrivateKey.generate()
        self.SPKb = X25519PrivateKey.generate()
        self.OPKb = X25519PrivateKey.generate()
        
    def __repr__(self): 
        #provides a way to print the public key values in base64 format.
        IKb_public_bytes = self.IKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        SPKb_public_bytes = self.SPKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        OPKb_public_bytes = self.OPKb.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        return "Bob Object:\n  Public Identity Key: %s \n  Public Signed Pre-Key: %s \n  Public Operational Key %s" % \
                (b64(IKb_public_bytes), b64(SPKb_public_bytes), b64(OPKb_public_bytes))


    def x3dh(self, alice):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        dh3 = self.SPKb.exchange(alice.EKa.public_key())
        dh4 = self.OPKb.exchange(alice.EKa.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Bob]\tShared key:', b64(self.sk))

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])

        
class Alice(object):
    def __init__(self):
        # generate Alice's keys
        self.IKa = X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()
        
    def __repr__(self): 
        #provides a way to print the public key values in base64 format.
        IKa_public_bytes = self.IKa.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        EKa_public_bytes = self.EKa.public_key().public_bytes(encoding=serialization.Encoding.Raw, format=serialization.PublicFormat.Raw)
        return "Alice Object:\n  Public Identity Key: %s \n  Public Ephemeral: %s \n" % \
                (b64(IKa_public_bytes), b64(EKa_public_bytes))


    def x3dh(self, bob):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.SPKb.public_key())
        dh4 = self.EKa.exchange(bob.OPKb.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Alice]\tShared key:', b64(self.sk))

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])


Instantiate Alice and Bob again, then print out the keys.  

In [15]:
alice = Alice()
bob = Bob()

print (alice)
print (bob)
print ()

# Alice performs an X3DH while Bob is offline, using his uploaded keys
alice.x3dh(bob)

# Bob comes online and performs an X3DH using Alice's public keys
bob.x3dh(alice)

Alice Object:
  Public Identity Key: 9RmM80H+zbizcAGSH9NLFGmMDrVOuFJWJEV+NBgtqEk= 
  Public Ephemeral: 7gNgo5FR2+q5C5kbEP/8QqprCLin5sAwS2eoL84yd38= 

Bob Object:
  Public Identity Key: c6KMcjiXtz1N5uca/BWtt7eO09EuliB2xLw9LgZmSns= 
  Public Signed Pre-Key: FUO+ZDfbVtoMpj6eUT85vDJI3Uo6NSvs0NeWnoyTUFg= 
  Public Operational Key 2l8reOAYlELMows0EFDzJ4MwMQ+pXC9IwT2g3rX452o=

[Alice]	Shared key: drwBMeK5b/i+YnP9WhDYDAWx8PQ6y3vY+zTVucIrK34=
[Bob]	Shared key: drwBMeK5b/i+YnP9WhDYDAWx8PQ6y3vY+zTVucIrK34=


Finally, initialize the ratchets and print out their states to show the matching states between the send and receive ratchets.

In [17]:
# Initialize their symmetric ratchets
alice.init_ratchets()
bob.init_ratchets()

# Print out the matching pairs
print('[Alice]\tsend ratchet:', list(map(b64, alice.send_ratchet.next())))
print('[Bob]\trecv ratchet:', list(map(b64, bob.recv_ratchet.next())))
print ()
print('[Alice]\trecv ratchet:', list(map(b64, alice.recv_ratchet.next())))
print('[Bob]\tsend ratchet:', list(map(b64, bob.send_ratchet.next())))


[Alice]	send ratchet: ['/JOFy701xEvLTr01Piiz9rjdktJoJ3Wkzrxgg9pqXyE=', 'NIxjs/4bUHF8Dcs5QxgKgg==']
[Bob]	recv ratchet: ['/JOFy701xEvLTr01Piiz9rjdktJoJ3Wkzrxgg9pqXyE=', 'NIxjs/4bUHF8Dcs5QxgKgg==']

[Alice]	recv ratchet: ['YBQ7shmOM+orwn5RtqF10Mqnid/YsCXEWnuvVg/TwTQ=', 'anBf2rEytHZwa/usLWrQ8w==']
[Bob]	send ratchet: ['YBQ7shmOM+orwn5RtqF10Mqnid/YsCXEWnuvVg/TwTQ=', 'anBf2rEytHZwa/usLWrQ8w==']


We can see their send and recv ratchets outputs match. This is of course because we initialized Alice’s sending ratchet first, but Bob’s second. This is not arbitrary - it depends on who sends the message as we’ll see in a bit, who we’ve been assuming is Alice in this case.

You might also observe there’s two outputs, while we expected one. That’s because the second one will be used as the IV for encrypting messages.

Alice and Bob have estabished a session now. They could now use these to send each other messages with proven confidentiality, integrity, authentication and forward secrecy, rotating the appropriate ratchet after sending or receiving a message. It does not however provide break-in recovery, as an attacker can still guess the future states of the sending or receiving ratchet if their state is leaked, or can deduce the states of both if the shared secret key of the root ratchet is leaked. This is why there’s our other type of ratchet: The Diffie-Hellman ratchet.

### Part 2b:  The Diffie-Hellman Ratchet

The second ratchet type is the Diffie-Hellman ratchet which is used to reset the keys used for the sending and receiving ratchets of both parties to new values.

Before receiving a message from Alice, Bob must initialize a new ratchet key pair RK_b and advertise the public key to Alice. Upon learning Bob’s public key, Alice will then generate her own key pair RK_a and calculate DH(RK_a, RK_b). This value will be used as the input to turn Alice’s root symmetric ratchet once, yielding a new key. This key will then be used to initialize Alice’s sending symmetric ratchet.

<img src="http://www.cs.colostate.edu/~cs456/images/dhRatchet.png" width="350px" alt="DH ratchet" />

Next, Bob can introduce a new key pair RK_b'. Using Alice’s previous public key he calculates DH(RK_b', RK_a) and uses that as input to his root chain to get a new key for initializing his sending ratchet. He can now discard his old key pair. When he sends his new message, encrypted with the sending ratchet, he advertises his new public key. Alice can then once again calculate DH(RK_a, RK_b') to update her receiving ratchet and decrypt Bob’s message, and can proceed with introducing her own new key pair.

Note that the input values for the sending and receiving ratchets are still constant. The input value for the root symmetric ratchet is, however, the output value of the DH ratchet. This way we now also ensure eventual break-in recovery: Even if the state of a ratchet is leaked to an attacker, we will soon afterwards perform a turn of the DH ratchet and the symmetric ratchets will be reset to new, unknown to the attacker values.

This process signifies a single turn of the DH ratchet, as in each step one party’s key is renewed and the old one is forgotten. It can be performed as often as the two parties like in order to provide break-in recovery. In practice it’s done with every single message.

With that, we can add the code for maintaining the DH ratchet by both Bob and Alice. We don’t need a new construct for the DH ratchet, as it’s sufficient to keep an X25519 key pair for each user.

In [18]:
class Bob(object):
    def __init__(self):
        # generate Bob's keys
        self.IKb = X25519PrivateKey.generate()
        self.SPKb = X25519PrivateKey.generate()
        self.OPKb = X25519PrivateKey.generate()
         # initialise Bob's DH ratchet
        self.DHratchet = X25519PrivateKey.generate()

    def x3dh(self, alice):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        dh3 = self.SPKb.exchange(alice.EKa.public_key())
        dh4 = self.OPKb.exchange(alice.EKa.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Bob]\tShared key:', b64(self.sk))


    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        
    def dh_ratchet(self, alice_public):
        # perform a DH ratchet rotation using Alice's public key
        dh_recv = self.DHratchet.exchange(alice_public)
        shared_recv = self.root_ratchet.next(dh_recv)[0]
        # use Alice's public and our old private key
        # to get a new recv ratchet
        self.recv_ratchet = SymmRatchet(shared_recv)
        print('[Bob]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Alice
        self.DHratchet = X25519PrivateKey.generate()
        dh_send = self.DHratchet.exchange(alice_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Bob]\tSend ratchet seed:', b64(shared_send))

class Alice(object):
    def __init__(self):
        # generate Alice's keys
        self.IKa = X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()
        # Alice's DH ratchet starts out uninitialised
        self.DHratchet = None

    def x3dh(self, bob):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.SPKb.public_key())
        dh4 = self.EKa.exchange(bob.OPKb.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Alice]\tShared key:', b64(self.sk))

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        
    def dh_ratchet(self, bob_public):
        # perform a DH ratchet rotation using Bob's public key
        if self.DHratchet is not None:
            # the first time we don't have a DH ratchet yet
            dh_recv = self.DHratchet.exchange(bob_public)
            shared_recv = self.root_ratchet.next(dh_recv)[0]
            # use Bob's public and our old private key
            # to get a new recv ratchet
            self.recv_ratchet = SymmRatchet(shared_recv)
            print('[Alice]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Bob
        self.DHratchet = X25519PrivateKey.generate()
        dh_send = self.DHratchet.exchange(bob_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Alice]\tSend ratchet seed:', b64(shared_send))


In [19]:
alice = Alice()
bob = Bob()

# Alice performs an X3DH while Bob is offline, using his uploaded keys
alice.x3dh(bob)

# Bob comes online and performs an X3DH using Alice's public keys
bob.x3dh(alice)

[Alice]	Shared key: Kb9eqGzKv7OCwVPFD1+iv62YlGbnksekufLV8kDf72c=
[Bob]	Shared key: Kb9eqGzKv7OCwVPFD1+iv62YlGbnksekufLV8kDf72c=


In [20]:
# Initialize their symmetric ratchets
alice.init_ratchets()
bob.init_ratchets()

# Initialize Alice's sending ratchet with Bob's public key
alice.dh_ratchet(bob.DHratchet.public_key())

[Alice]	Send ratchet seed: Zk5k8eojKi6xVS4KVTFL26tT+OWGaDSB8DxlPUZl1VM=


Bob can’t yet however decrypt Alice’s message! He also needs to turn his own DH ratchet, and that depends on Alice’s public key, which she must send along with her message. Let’s implement both their send and recv methods.

We will use symmetric encryption using the AES standard cipher to do the encryption using the ratchet-generated keys.

In [21]:
def pad(msg):
    # pkcs7 padding
    num = 16 - (len(msg) % 16)
    return msg + bytes([num] * num)

def unpad(msg):
    # remove pkcs7 padding
    return msg[:-msg[-1]]

class Bob(object):
    def __init__(self):
        # generate Bob's keys
        self.IKb = X25519PrivateKey.generate()
        self.SPKb = X25519PrivateKey.generate()
        self.OPKb = X25519PrivateKey.generate()
         # initialise Bob's DH ratchet
        self.DHratchet = X25519PrivateKey.generate()

    def x3dh(self, alice):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.SPKb.exchange(alice.IKa.public_key())
        dh2 = self.IKb.exchange(alice.EKa.public_key())
        dh3 = self.SPKb.exchange(alice.EKa.public_key())
        dh4 = self.OPKb.exchange(alice.EKa.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Bob]\tShared key:', b64(self.sk))


    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        
    def dh_ratchet(self, alice_public):
        # perform a DH ratchet rotation using Alice's public key
        dh_recv = self.DHratchet.exchange(alice_public)
        shared_recv = self.root_ratchet.next(dh_recv)[0]
        # use Alice's public and our old private key
        # to get a new recv ratchet
        self.recv_ratchet = SymmRatchet(shared_recv)
        print('[Bob]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Alice
        self.DHratchet = X25519PrivateKey.generate()
        dh_send = self.DHratchet.exchange(alice_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Bob]\tSend ratchet seed:', b64(shared_send))
        
    def send(self, alice, msg):
        key, iv = self.send_ratchet.next()
        cipher = AES.new(key, AES.MODE_CBC, iv).encrypt(pad(msg))
        print('[Bob]\tAES Symmetric Key used to encrypt message: ', b64(iv))
        print('[Bob]\tSending ciphertext to Alice:', b64(cipher))
        # send ciphertext and current DH public key
        alice.recv(cipher, self.DHratchet.public_key())

    def recv(self, cipher, alice_public_key):
        # receive Alice's new public key and use it to perform a DH
        self.dh_ratchet(alice_public_key)
        key, iv = self.recv_ratchet.next()
        # decrypt the message using the new recv ratchet
        msg = unpad(AES.new(key, AES.MODE_CBC, iv).decrypt(cipher))
        print('[Bob]\tAES Symmetric Key used to decrypt message: ', b64(iv))
        print('[Bob]\tDecrypted message:', msg)

class Alice(object):
    def __init__(self):
        # generate Alice's keys
        self.IKa = X25519PrivateKey.generate()
        self.EKa = X25519PrivateKey.generate()
        # Alice's DH ratchet starts out uninitialised
        self.DHratchet = None

    def x3dh(self, bob):
        # perform the 4 Diffie Hellman exchanges (X3DH)
        dh1 = self.IKa.exchange(bob.SPKb.public_key())
        dh2 = self.EKa.exchange(bob.IKb.public_key())
        dh3 = self.EKa.exchange(bob.SPKb.public_key())
        dh4 = self.EKa.exchange(bob.OPKb.public_key())
        # the shared key is KDF(DH1||DH2||DH3||DH4)
        self.sk = hkdf(dh1 + dh2 + dh3 + dh4, 32)
        print('[Alice]\tShared key:', b64(self.sk))

    def init_ratchets(self):
        # initialise the root chain with the shared key
        self.root_ratchet = SymmRatchet(self.sk)
        # initialise the sending and recving chains
        self.send_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        self.recv_ratchet = SymmRatchet(self.root_ratchet.next()[0])
        
    def dh_ratchet(self, bob_public):
        # perform a DH ratchet rotation using Bob's public key
        if self.DHratchet is not None:
            # the first time we don't have a DH ratchet yet
            dh_recv = self.DHratchet.exchange(bob_public)
            shared_recv = self.root_ratchet.next(dh_recv)[0]
            # use Bob's public and our old private key
            # to get a new recv ratchet
            self.recv_ratchet = SymmRatchet(shared_recv)
            print('[Alice]\tRecv ratchet seed:', b64(shared_recv))
        # generate a new key pair and send ratchet
        # our new public key will be sent with the next message to Bob
        self.DHratchet = X25519PrivateKey.generate()
        dh_send = self.DHratchet.exchange(bob_public)
        shared_send = self.root_ratchet.next(dh_send)[0]
        self.send_ratchet = SymmRatchet(shared_send)
        print('[Alice]\tSend ratchet seed:', b64(shared_send))
        
    def send(self, bob, msg):
        key, iv = self.send_ratchet.next()
        cipher = AES.new(key, AES.MODE_CBC, iv).encrypt(pad(msg))
        print ('[Alice]\tAES Symmetric Key used to encrypt message: ', b64(iv))
        print('[Alice]\tSending ciphertext to Bob:', b64(cipher))
        # send ciphertext and current DH public key
        bob.recv(cipher, self.DHratchet.public_key())

    def recv(self, cipher, bob_public_key):
        # receive Bob's new public key and use it to perform a DH
        self.dh_ratchet(bob_public_key)
        key, iv = self.recv_ratchet.next()
        # decrypt the message using the new recv ratchet
        msg = unpad(AES.new(key, AES.MODE_CBC, iv).decrypt(cipher))
        print('[Alice]\tAES Symmetric Key used to decrypt message: ', b64(iv))
        print('[Alice]\tDecrypted message:', msg)

and now with the class structures finalized, let's send a message!

In [22]:

alice = Alice()
bob = Bob()

# Alice performs an X3DH while Bob is offline, using his uploaded keys
alice.x3dh(bob)

# Bob comes online and performs an X3DH using Alice's public keys
bob.x3dh(alice)

# Initialize their symmetric ratchets
alice.init_ratchets()
bob.init_ratchets()

# Initialise Alice's sending ratchet with Bob's public key
alice.dh_ratchet(bob.DHratchet.public_key())

# Alice sends Bob a message and her new DH ratchet public key
alice.send(bob, b'Hello Bob!')

# Bob uses that information to sync with Alice and send her a message
bob.send(alice, b'Hello to you too, Alice!')

[Alice]	Shared key: GNEIMvLcyaWZAOh7wPRA9A4FijLjpwgW/IX+v0s/k/s=
[Bob]	Shared key: GNEIMvLcyaWZAOh7wPRA9A4FijLjpwgW/IX+v0s/k/s=
[Alice]	Send ratchet seed: aXOm4UnULWQa9CTqA+97jFGpO1Z+0e6OzwIUNvZWqFM=
[Alice]	AES Symmetric Key used to encrypt message:  iCRkrAjj+1cLlITrd3TAow==
[Alice]	Sending ciphertext to Bob: /tTFnt1/Ui+9WqWZx2YVBw==
[Bob]	Recv ratchet seed: aXOm4UnULWQa9CTqA+97jFGpO1Z+0e6OzwIUNvZWqFM=
[Bob]	Send ratchet seed: RSuanXid+MJzDv8+vFKO0mXwmMz6kzrpiwEssBU+F4c=
[Bob]	AES Symmetric Key used to decrypt message:  iCRkrAjj+1cLlITrd3TAow==
[Bob]	Decrypted message: b'Hello Bob!'
[Bob]	AES Symmetric Key used to encrypt message:  K3auDCJzuef979PQTSfCJg==
[Bob]	Sending ciphertext to Alice: 1QemJsmOIsSjt42u0ogvE/GyGwfRF1VmICCqWtfaUqE=
[Alice]	Recv ratchet seed: RSuanXid+MJzDv8+vFKO0mXwmMz6kzrpiwEssBU+F4c=
[Alice]	Send ratchet seed: INZMldxVrXWSm+40MmcVoPGiZmP1C684OFWHkPiQFxc=
[Alice]	AES Symmetric Key used to decrypt message:  K3auDCJzuef979PQTSfCJg==
[Alice]	Decrypted message: b'Hell

#  <font color='red'> Exercise #1 </font>

## Your Turn:  send a message that includes your name

In the cell below, send the following message from Alice to Bob 

    " This message is created by FIRSTNAME LASTNAME", substituting your name for FIRSTNAME LASTNAME

Then send a reply from Bob with the following message:

    "Thank you LASTNAME" substituting your last name for LASTNAME

__1)__ Send same message and same reply and watch the seeds and AES keys.

__2)__ Send another message and reply and watch the seeds and AES keys.


In [34]:
# Alice sends Bob a message and her new DH ratchet public key
alice.send(bob, b'This message is create by AIDAN MICHALOS,')

# Bob uses that information to sync with Alice and send her a message
bob.send(alice, b'Hello to you too, Alice!')

[Alice]	AES Symmetric Key used to encrypt message:  DSTQPK2FgQhWmqIMJ0OlAw==
[Alice]	Sending ciphertext to Bob: GoJG9bvXq3IUlRNdRZZfWhrtkeC0chBNi9vjkojmZubNWa2GM3tLDdb3M+zMkoEM
[Bob]	Recv ratchet seed: SJZACrwsZz0/jWI/Hxy7kXMuKoZXd4swNY7BrnyE/8o=
[Bob]	Send ratchet seed: OeWGbzUIRip2aNrHHJ00vupfrgCDYVaU0QmPydtMt0Y=
[Bob]	AES Symmetric Key used to decrypt message:  DSTQPK2FgQhWmqIMJ0OlAw==
[Bob]	Decrypted message: b'This message is created by AIDAN MICHALOS'
[Bob]	AES Symmetric Key used to encrypt message:  e6e36d266FRva55lNS90Ew==
[Bob]	Sending ciphertext to Alice: rnj/wk0pTY+meNRJ4/xvgnRHNGXwaASIPAn0QMxy5gg=
[Alice]	Recv ratchet seed: OeWGbzUIRip2aNrHHJ00vupfrgCDYVaU0QmPydtMt0Y=
[Alice]	Send ratchet seed: aI+XD3Lvzujo9ta4RNt9l+CbLi+zaSXJC5alHwaUzSU=
[Alice]	AES Symmetric Key used to decrypt message:  e6e36d266FRva55lNS90Ew==
[Alice]	Decrypted message: b'Thank you MICHALOS'


## What to submit to Canvas

Copy the __code__ and the __cell output__ of your work from the cell above showing the keys and the messages.  

A) Paste the code and the output into a text file or word document and submit this file to canvas.  
B) write a brief paragraph with your observations about the seeds and the AES keys for items 1 and 2 above.
